In [ ]:
!wget https://raw.githubusercontent.com/HenriqueSabino/Localization-mod-template-creator/main/ModTemplate.template

#### Creating the main mod file

In [7]:
import glob
import pandas as pd
import json
import os

mod_template = open("./ModTemplate.template", "r", encoding="utf-8")

mod_class_name = input("Enter the desired class name: ")
mod_guid = input("Enter the desired mod GUID: ")
mod_title = input("Enter the desired mod title: ")
mod_description = input("Enter the desired mod description: ")

template_text = "".join(mod_template.readlines())
mod_template.close()

output_text = template_text.replace("{mod_class_name}", mod_class_name)
output_text = output_text.replace("{mod_guid}", mod_guid)
output_text = output_text.replace("{mod_title}", mod_title)
output_text = output_text.replace("{mod_description}", mod_description)

translation_files = glob.glob("*.csv")
translation_tables = [x.split('.')[0] for x in translation_files]

localization_methods_calls = ""
for file in translation_tables:
    file_no_spaces = file.replace(" ", "")
    localization_methods_calls += f"this.Set{file_no_spaces}Localizations();\n            "

output_text = output_text.replace("{localization_methods_calls}", localization_methods_calls.strip())

localization_methods = ""
for file in translation_files:
    translation = pd.read_csv(f"./{file}")

    file_no_extension = file.split(".")[0]
    file_no_spaces = file_no_extension.replace(" ", "")

    localization_methods += f"private void Set{file_no_spaces}Localizations()\n        "
    localization_methods += "{\n            "
    localization_methods += f"var table = LocalizationHelper.GetCollection(\"{file_no_extension}\", ModLanguage);\n            "
    localization_methods += "\n            "

    for entry in translation.itertuples():

        localized_text = json.dumps(entry.Value, ensure_ascii=False)

        if localized_text == "NaN":
            localized_text = "string.Empty"

        localization_methods += f"table.AddEntry(\"{entry.Key}\", {localized_text});\n            "

    localization_methods = localization_methods.strip() + "\n        }\n\n        "

output_text = output_text.replace("{localization_methods}", localization_methods.strip()).strip()

try:
    os.mkdir("./out")
except:
    print("out folder already exists")

output_file = open(f"./out/{mod_class_name}.cs", "w", encoding="utf-8")
output_file.write(output_text)
output_file.close()

#### Creating the patches

In [8]:
patch_files = glob.glob("*Patch.template")

try:
    os.mkdir("./out/Patches")
except:
    print("Patches folder already created")

for patch in patch_files:

    patch_template = open(patch, "r", encoding="utf-8")
    patch_text = template_text = "".join(patch_template.readlines())
    patch_text = patch_text.replace("{mod_class_name}", mod_class_name)

    patch_template.close()

    patch_name = patch.replace("template", "cs")
    patch_output = open(f"./out/Patches/{patch_name}", "w", encoding="utf-8")
    patch_output.write(patch_text)
    patch_output.close()

    

ChineseCollectionPatch.template
ClassesGetterPatch.template
LocalizerFormatterPatch.template
SpriteGlyphPatch.template
